
# Final Assignment — Predicting Wine Quality with Linear Regression

This notebook follows the assignment structure:
1) **Dataset Description**  
2) **Analysis Objective**  
3) **Model Comparison**  
4) **Findings & Interpretation**  
5) **Evaluation & Future Steps**  


## 1) Dataset Description — Setup & Load Data

In [ ]:

# --- Install dependencies (safe to re-run) ---
# If your environment already has these, pip will skip or reuse cached wheels.
!pip -q install pandas numpy matplotlib scikit-learn wget


In [ ]:

# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import wget


In [ ]:

# --- Download the Wine Quality (red) dataset from UCI ---
url_red = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wget.download(url_red, "winequality-red.csv")
print("\nDownloaded winequality-red.csv")


In [ ]:

# --- Load Data ---
df = pd.read_csv("winequality-red.csv", sep=";")
print("✅ Data loaded:", df.shape)
df.head()


In [ ]:

# --- Structure / Summary ---
print("Info:")
df.info()
print("\nMissing values per column:")
print(df.isnull().sum())
display(df.describe())


In [ ]:

# --- Correlation matrix (matplotlib only) ---
corr = df.corr(numeric_only=True)
fig = plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest')
plt.title("Correlation Matrix of Wine Features")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.colorbar()
plt.tight_layout()
plt.show()



## 2) Analysis Objective

**Goal:** Predict **wine quality** (0–10) from physicochemical attributes using linear models, and compare
- Ordinary Least Squares (Linear Regression)
- Ridge Regression (L2)
- LASSO Regression (L1)

**Success criteria:** Higher test R² with lower RMSE/MAE while keeping the model simple (avoiding overfitting).


## 3) Model Comparison — Train, Evaluate, Compare

In [ ]:

# --- Train / Test Split ---
X = df.drop("quality", axis=1)
y = df["quality"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Scale Features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define Models ---
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.01, random_state=42, max_iter=10000)
}

# --- Fit & Evaluate ---
import math

results = {}
preds = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    preds[name] = y_pred
    results[name] = {
        "R2": r2_score(y_test, y_pred),
        "RMSE": math.sqrt(mean_squared_error(y_test, y_pred)),
        "MAE": mean_absolute_error(y_test, y_pred)
    }

results_df = pd.DataFrame(results).T.sort_values("R2", ascending=False)
print("Model performance on test set:")
display(results_df)


## 4) Findings & Interpretation

In [ ]:

# --- Coefficient comparison plot (matplotlib only) ---
coef_table = pd.DataFrame({
    "Feature": X.columns,
    "Linear": models["Linear"].coef_,
    "Ridge": models["Ridge"].coef_,
    "Lasso": models["Lasso"].coef_
})

coef_table = coef_table.set_index("Feature")
ax = coef_table.plot(kind="bar", figsize=(12,5))
plt.title("Model Coefficients Across Linear, Ridge, and Lasso")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# --- Brief textual insights (printed) ---
top_corr = df.corr(numeric_only=True)["quality"].abs().sort_values(ascending=False).index[:5].tolist()
print("Top 5 absolute correlations with quality:", top_corr)
best_model = results_df.index[0]
print(f"Best model by R² on the test set: {best_model}")


## 5) Evaluation & Future Steps

In [ ]:

print("""Evaluation:
- Linear models show moderate predictive power; quality is only partly explained by physicochemical variables.
- Regularization (Ridge/Lasso) reduces variance; Lasso may zero small coefficients for a simpler model.

Limitations:
- Assumes linear relationships and no strong interactions.
- 'quality' is ordinal but treated as continuous.
- No domain/categorical features (e.g., grape, region, vintage).

Next Steps:
- Try polynomial features or tree-based models (Random Forest, Gradient Boosting).
- Explore interaction terms (e.g., acidity * alcohol).
- Evaluate with cross-validation and hyperparameter tuning (GridSearchCV for alphas).
- Consider ordinal regression or a classification framing (quality buckets).
""")
